# Quality Control of RNA-Seq Data from Olympia Oysters
Quality control, filtering, and trimming of RNA-Seq reads from a in preparation for Trinity assembly. Following along with [this tutorial](https://informatics.fas.harvard.edu/best-practices-for-de-novo-transcriptome-assembly-with-trinity.html). Data was received on Feb. 1, 2018 and consists of 11 samples sequenced on 2 PE 100bp Illumina HiSeq4000 lanes at the UChicago Functional Genomics Core. samples from a low pH challenge, primarily from three populations and multiple tissue types. RNA extraction protocol and information found [here](https://benchling.com/s/etr-5HBBfmBYAvQbzdrHjc61).
#### Sample Information:
| FastQ File Name | Sample Name | Tissue  | Treatment| Time Point|    
| :------------- |:-------------| :-------|:-------- |:--------- |
| CP-1_S1       | CA1B_14A        | Adductor| Low pH   | T1 |
| CP-2_S2      | OR1A_1C     |   Ctenidia |  Ambient | T1 |
| CP-3_S3 |      BC1A_9C      |    Ctenidia | Ambient | T1 |
| CP-4_S4 |   CA1A_6M         | Mantle | Ambient | T1 |
|CP-5_S5 |    CA1A_6C          | Ctenidia | Ambient | T1 |
| CP-6_S6 |   CA1A_6A          | Adductor | Ambient | T1 |
|CP-15_S7 |   OR1B_24C    | Ctenidia | Low pH | T1 |
| CP-16_S8 |  BC1B_16C    | Ctenidia | Low pH | T1 |
| CP-17_S9 |   CA1B_14M   | Mantle | Low pH | T1 |
| CP-18_S10 |   CA1B_14C  | Ctenidia | Low pH | T1 |
| CP-4Spl_S11 | SN_20_ML,SN_31_FA,NF_23_FL,NF_28_MA | Gonad pool | Low and Ambient | Roberts Exp |


In [1]:
%pwd

u'/home/t.cri.ksilliman/OA_RNA'

Rename files so the two lanes are kept separate, then move into 1 folder.

In [ ]:
%%bash
for i in /scratch/t.cri.ksilliman/OA_RNA/Oyster/Oyster_Raw_RNASeq/0343/FastQ/*.gz;
    do mv $i ${i/001/0343};
done;

for i in /scratch/t.cri.ksilliman/OA_RNA/Oyster/Oyster_Raw_RNASeq/0348/FastQ/*.gz;
    do mv $i ${i/001/0348};
done;

In [ ]:
%%sh
mv /scratch/t.cri.ksilliman/OA_RNA/Oyster/Oyster_Raw_RNASeq/0343/FastQ/* /scratch/t.cri.ksilliman/OA_RNA/Oyster/Oyster_Raw_RNASeq/
mv /scratch/t.cri.ksilliman/OA_RNA/Oyster/Oyster_Raw_RNASeq/0348/FastQ/* /scratch/t.cri.ksilliman/OA_RNA/Oyster/Oyster_Raw_RNASeq/

### FastQC (summarize using [MultiQC](http://multiqc.info/))

In [ ]:
%%sh
module load java-jdk/1.8.0_92
module load fastqc/0.11.5

fastqc -o /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/ --extract /scratch/t.cri.ksilliman/OA_RNA/Oyster/Oyster_Raw_RNASeq/*.fastq.gz -t 8

In [ ]:
%%sh
multiqc /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output -o /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output

In [ ]:
from IPython.core.display import display, HTML
display(HTML('/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/multiqc_report.html'))

### Removing erroneous k-mers from Illumina paired-end reads ([Rcorrector](https://github.com/mourisl/Rcorrector))
"Because rare k-mers are likely due to sequencing errors, correcting reads such that rare k-mers are corrected to a more frequently occurring can improve assemblies."

In [ ]:
%%sh
# For CRI, make sure you build Rcorrector with jellyfish and zlib modules loaded, 
# and then load those modules whenever you run Rcorrector
module load gcc/6.2.0
module load jellyfish/2.2.3
module load zlib/1.2.8
# Rcorrector creates temporary files in the directory it is run, so move to /scratch before running
cd /scratch/t.cri.ksilliman/OA_RNA/

for i in /scratch/t.cri.ksilliman/OA_RNA/Scallop/Oyster_Raw_RNASeq/*R1*.fastq.gz;
    do perl ~/Downloads/rcorrector/run_rcorrector.pl -t 10 -1 $i -2 ${i/R1/R2} \
    -od /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/;
done;

Output fastq files will have "cor" in name.

### Discard read pairs with an unfixable read ([Harvard FAS Script](https://github.com/harvardinformatics/TranscriptomeAssemblyTools/blob/master/FilterUncorrectabledPEfastq.py))  
Rcorrector flags reads with erroneous kmers that can't be fixed. These often have lots of Ns or are low complexity "junk" sequences. This python script identifies read pairs for which at least one read is "unfixeable" and removes them, as well as removing the tags created by Rcorrector in the sequence headers.

In [ ]:
%%sh

for s in /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/*R1*cor*;
    do python FilterUncorrectabledPEfastq.py --left_reads $s --right_reads ${s/R1/R2} --out_prefix /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/fixed;
done;
;

### Trim adapter and low quality bases ([Trim Galore!](https://www.bioinformatics.babraham.ac.uk/projects/trim_galore/))
Recommended over Trimmomatic as it performs adapter removal better. Requires Cutadapt, which I installed on CRI using Miniconda. This is the most customizeable filtering/QC step. I'm using defaults recommended by Harvard FAS.

In [ ]:
%%sh

for s in /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/fixed*R1*;
    do /home/t.cri.ksilliman/TrimGalore-0.4.5/trim_galore --paired --retain_unpaired --phred33 \
    --output_dir /scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads --length 36 -q 5 \
    --stringency 1 -e 0.1 $s ${s/R1/R2} > /scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/TG.stats;
done;

###  Map trimmed reads to a blacklist to remove unwanted (rRNA reads)
Using a rRNA database from [Silva](https://www.arb-silva.de/) and Bowtie2.

In [24]:
%%sh
wget https://www.arb-silva.de/fileadmin/silva_databases/current/Exports/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta.gz \
    -P /scratch/t.cri.ksilliman/OA_RNA/

gunzip /scratch/t.cri.ksilliman/OA_RNA/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta.gz

--2018-02-11 11:41:49--  https://www.arb-silva.de/fileadmin/silva_databases/current/Exports/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta.gz
Resolving www.arb-silva.de... 134.102.40.6
Connecting to www.arb-silva.de|134.102.40.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238828503 (228M) [application/gzip]
Saving to: `/scratch/t.cri.ksilliman/OA_RNA/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta.gz'

     0K .......... .......... .......... .......... ..........  0%  152K 25m34s
    50K .......... .......... .......... .......... ..........  0%  219K 21m40s
   100K .......... .......... .......... .......... ..........  0%  436K 17m25s
   150K .......... .......... .......... .......... ..........  0%  437K 15m17s
   200K .......... .......... .......... .......... ..........  0%  435K 14m0s
   250K .......... .......... .......... .......... ..........  0%  443K 13m8s
   300K .......... .......... .......... .......... ..........  0% 24.9M 11m16s
   350K .....

In [25]:
%%sh
module load intel/2017
module load bowtie2/2.3.2

bowtie2-build /scratch/t.cri.ksilliman/OA_RNA/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta /scratch/t.cri.ksilliman/OA_RNA/rRNA_ref

Settings:
  Output files: "/scratch/t.cri.ksilliman/OA_RNA/rRNA_ref.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /scratch/t.cri.ksilliman/OA_RNA/SILVA_132_SSURef_Nr99_tax_silva_trunc.fasta
Reading reference sizes
  Time reading reference sizes: 00:00:15
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:09
bmax according to bmaxDivN setting: 198441567
Using parameters --bmax 148831176 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing wit

Building a SMALL index


In [29]:
%%sh
#Change file names so they don't end in _1 and _2 of started in fixed
for r in /scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/*R1*fq; do
    mv "$r" "$(echo $r | sed 's/fixed_CP/CP/g; s/_1.fq/.fq/g')";
done

for r in /scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/*R2*fq; do
    mv "$r" "$(echo $r | sed 's/fixed_CP/CP/g; s/_2.fq/.fq/g')";
done

In [3]:
%%sh
cd /scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/
for file in *R1_0343*val.fq;
    do echo "${file/_L004_R1_0343.cor_val.fq/_}" >> Samples.txt;
done;
cat Samples.txt

CP-15_S7_
CP-16_S8_
CP-17_S9_
CP-18_S10_
CP-1_S1_
CP-2_S2_
CP-3_S3_
CP-4Spl_S11_
CP-4_S4_
CP-5_S5_
CP-6_S6_


Want to keep the $F reads.

In [ ]:
%%sh
module load intel/2017
module load bowtie2/2.3.2

mkdir /scratch/t.cri.ksilliman/OA_RNA/Oyster/filtered_RNASeq/
#0343 Lane
A="/scratch/t.cri.ksilliman/OA_RNA/rRNA_ref"
while read file; do
    B="/scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/${file}L004_R1_0343.cor_val.fq"
    C="/scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/${file}L004_R2_0343.cor_val.fq"
    D="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0343_rRNASum.txt"
    E="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0343_rPEmapped.fq.gz"
    F="/scratch/t.cri.ksilliman/OA_RNA/Oyster/filtered_RNASeq/${file}0343_filt.fq.gz"
    G="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0343_rSEmapped.fq.gz"
    H="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0343_rSEunmapped.fq.gz"
    bowtie2 --very-sensitive-local --phred33  -x $A -1 $B -2 $C --threads 10 --met-file $D --al-conc-gz $E --un-conc-gz $F --al-gz $G --un-gz $H;
done < /scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/Samples.txt

In [ ]:
%%sh
#0348 Lane
module load intel/2017
module load bowtie2/2.3.2

A="/scratch/t.cri.ksilliman/OA_RNA/rRNA_ref"
while read file; do
    B="/scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/${file}L004_R1_0348.cor_val.fq"
    C="/scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/${file}L004_R2_0348.cor_val.fq"
    D="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0348_rRNASum.txt"
    E="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0348_rPEmapped.fq.gz"
    F="/scratch/t.cri.ksilliman/OA_RNA/Oyster/filtered_RNASeq/${file}0348_filt.fq.gz"
    G="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0348_rSEmapped.fq.gz"
    H="/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/${file}0348_rSEunmapped.fq.gz"
    bowtie2 --very-sensitive-local --phred33  -x $A -1 $B -2 $C --threads 10 --met-file $D --al-conc-gz $E --un-conc-gz $F --al-gz $G --un-gz $H;
done < /scratch/t.cri.ksilliman/OA_RNA/Oyster/trimmed_reads/Samples.txt

### Check filtered and trimmed reads with FastQC

In [ ]:
%%sh
module load java-jdk/1.8.0_92
module load fastqc/0.11.5

mkdir /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/filtered_QC/

fastqc -o /scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/filtered_QC --extract /scratch/t.cri.ksilliman/OA_RNA/Oyster/filtered_RNASeq/*.fq.gz -t 8

In [ ]:
from IPython.core.display import display, HTML
display(HTML('/scratch/t.cri.ksilliman/OA_RNA/Oyster/QC_Output/filtered_QC/multiqc_report.html'))